<a href="https://colab.research.google.com/github/SHADAMAKINIKHITHA/Water-Quality/blob/master/Banking%20Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# used a dictionary to represent an intents JSON file
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "How are you?", "Hi there", "Hi"],
              "responses": ["Hii Partner!", "Hello", "Greetings!","Iam fine...","Hi"],
             },
             {"tag": "age",
              "patterns": ["how old are you?", "when is your birthday?", "when were you born?","when was you born?","what is your age?","Your Birth Date/Day?"],
              "responses": ["I am 24 years old", "I was born in 1996", "My birthday is July 3rd and I was born in 1996", "03/07/1996"]
             },
             {"tag": "date",
              "patterns": ["what are you doing this weekend?","do you want to hang out some time?", "what are your plans for this week"],
              "responses": ["I am available all week", "I don't have any plans", "I am not busy"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "what are you called?", "who are you?","By which name you are called?"],
              "responses": ["My name is Kippi", "I'm Kippi", "Kippi"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "see ya", "adios", "cya"],
              "responses": ["It was nice speaking to you", "See you later", "Speak soon!"]
             }
]}

In [ ]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
print(words)

["'s", 'adios', 'age', 'are', 'birth', 'birthday', 'born', 'by', 'bye', 'called', 'cya', 'date/day', 'do', 'doing', 'for', 'hang', 'hello', 'hi', 'how', 'is', 'name', 'old', 'out', 'plan', 'see', 'some', 'there', 'this', 'time', 'to', 'wa', 'want', 'week', 'weekend', 'were', 'what', 'when', 'which', 'who', 'ya', 'you', 'your']


In [ ]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [ ]:
print(doc_X)

['Hello', 'How are you?', 'Hi there', 'Hi', 'how old are you?', 'when is your birthday?', 'when were you born?', 'when was you born?', 'what is your age?', 'Your Birth Date/Day?', 'what are you doing this weekend?', 'do you want to hang out some time?', 'what are your plans for this week', "what's your name?", 'what are you called?', 'who are you?', 'By which name you are called?', 'bye', 'see ya', 'adios', 'cya']


In [ ]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


In [ ]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               5504      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 14,085
Trainable params: 14,085
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [============================

In [ ]:


def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
#.................................................................
def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# running the chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

hii
Greetings!
how are you
Greetings!
whats your age
My birthday is July 3rd and I was born in 1996
age
I was born in 1996
age 
My birthday is July 3rd and I was born in 1996
how old are you
03/07/1996
how old are you
03/07/1996
how old are you
03/07/1996
how old are you?
03/07/1996
birth date
I am 24 years old
birth day
My birthday is July 3rd and I was born in 1996
old are you
I was born in 1996
whats your name
My name is Kippi
what should i call you
I'm Kippi
name of you
Kippi
hello
Hii Partner!
age
I am 24 years old
age
My birthday is July 3rd and I was born in 1996
age
I am 24 years old
okay
Hii Partner!
ok
Iam fine...
byee
Iam fine...
bye
See you later
eeee
Hi
